In [346]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of cig_script.mail_copy_input_format failed: Traceback (most recent call last):
  File "c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "c:\users\user\appdata\local\programs\python\python38\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\USER\Desktop\CIG\cig_script\mail_copy_input_format.py", line 4, in <module>
    from config.config_env import Config
ModuleNotFoundError: No module named 'config'
]


In [347]:
import os
import pandas as pd
import numpy as np
import sys
import re
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

pd.set_option('display.max_columns', 50)

In [348]:
from cig_script.config.config_env import Config
from cig_script.mail_copy_input_format import _get_mul_car_mail_in_format
from cig_script.mail_copy_input_format import _remove_NBSP_df
from cig_script.mail_copy_input_format import _reset_index
from cig_script.mail_copy_input_format import _get_dict_description_info
from cig_script.mail_copy_input_format import _get_mul_car_info_df
from cig_script.mail_copy_input_format import _merge_df_description_info





In [349]:
mail_copy_in_path = Config().mail_copy_in_path
df = _get_mul_car_mail_in_format(mail_copy_in_path)
df = _remove_NBSP_df(df)
df[0].head(13)
temp_df = df.loc[:,[0]]
temp_df = temp_df.dropna()
temp_df = _reset_index(temp_df)


In [350]:
def _get_df_mul_car_info(con_df,cate,val,cnt):
    dic = {}
    dic.update({cate:{cnt:val}})
    df = pd.DataFrame.from_dict(data=dic)
    
    con_df = pd.concat([con_df,df])
    return con_df

In [351]:
con_dict = {}
BL_cnt = 0
con_name_list = []
con_df = pd.DataFrame()
acid_df = pd.DataFrame()
import_tax_df = pd.DataFrame()
export_num_df = pd.DataFrame()

for idx,col in temp_df.iterrows():
    col = col.str.upper()
    val = col[0]
    
    if 'CONSIGNEE' in val:
        BL_cnt = BL_cnt + 1
        if idx >= 1:
            consign_idx = idx + 1
            cate = val
            con_name = temp_df.loc[consign_idx,:][0]
            con_df = _get_df_mul_car_info(con_df,cate,con_name,BL_cnt)
            
    elif 'ACID' in val:
            acid_no_list = val.split(':')
            cate = acid_no_list[0]
            acid_no = acid_no_list[1]
            acid_df = _get_df_mul_car_info(acid_df,cate,acid_no,BL_cnt)
    
    elif 'IMPORTER VAT NUMBER' in val or 'IMPORTER TAX NUMBER' in val:
            import_tax_list = val.split(':')
            cate = import_tax_list[0]
            import_tax = import_tax_list[1]
            import_tax_df = _get_df_mul_car_info(import_tax_df,cate,import_tax,BL_cnt)
    
    elif 'EXPORTER REGISTRATION NUMBER' in val:
            export_num_list = val.split(':')
            cate = export_num_list[0]
            export_num = export_num_list[1]
            export_num_df = _get_df_mul_car_info(export_num_df,cate,export_num,BL_cnt)

car_info_df = _get_mul_car_info_df(df)
total_df = _merge_df_description_info(con_df,acid_df,import_tax_df,export_num_df,car_info_df)

total_df

,CONSIGNEE NAME:,ACID NO,EGYPTIAN IMPORTER VAT NUMBER,FOREIGN EXPORTER REGISTRATION NUMBER,MODEL,YR,CHASSINO
index,,,,,,,
1,MIDDLE EAST TRADING CENTER ( ELRAWAS),2049059822022070346,204905982,6348801265,"{""1"": ""AVANTE"", ""2"": ""AVANTE"", ""3"": ""AVANTE"", ...","{""1"": 2011.0, ""2"": 2011.0, ""3"": 2011.0, ""4"": 2...","{""1"": ""KMHDH41DBBU044101"", ""2"": ""KMHDH41DBBU06..."
2,TRANS WORLD TRADING CO. ( ELROTOB BROTHERS),2049062532022080021,204906253,6348801265,"{""1"": ""AVANTE"", ""2"": ""AVANTE"", ""3"": ""AVANTE"", ...","{""1"": 2012.0, ""2"": 2011.0, ""3"": 2007.0, ""4"": 2...","{""1"": ""KMHDG41DBCU498818"", ""2"": ""KMHDH41DBBU03..."
